In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from copy import deepcopy

df=pd.read_csv('C:/Users/cash/Desktop/Mobilis/mobilis_django/filtered_data.csv')

In [ ]:

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371
    return c * r

def Estimate_Time(start, end,speed_kmh):
    lon1, lat1 = start['Longitude'], start['Latitude']
    lon2, lat2 = end['Longitude'], end['Latitude']
    
    distance_km = haversine(lon1, lat1, lon2, lat2)
    
    
    time_hours = distance_km / speed_kmh
    time_minutes = time_hours * 60
    
    return {
        'distance_km': round(distance_km, 2),
        'estimated_time_minutes': round(time_minutes, 2)
    }


start = {'Longitude': 2.033, 'Latitude': 36.752}
end = {'Longitude': 3.0588, 'Latitude': 36.7538}
print(Estimate_Time(start, end,49))


In [ ]:
def print_pretty_result(result):
    if result['points_visited']<result['Original_points']+1:
        print(f'Visiting All the points In the time limit is unlikely to be optimal. You can try to increase the time limit or average speed.')
    print(f"\n📍 Points Visited: {result['points_visited']}")
    print(f"⏱️ Total Travel Time: {result['total_time_minutes']} minutes")
    print("\n🗺️ Visit Order:")
    for i, point in enumerate(result['visited_points'], start=1):
        name = point.get('name', f"Unnamed ({point['Latitude']}, {point['Longitude']})")
        print(f"  {i}. {name}")

In [ ]:


class PlanOptimizerFull(object):
    def __init__(self, data, time_limit, constraints=None, speed=40, daily_limit=480):
        self.original_data = deepcopy(data)
        self.data = deepcopy(data)
        self.global_time_limit = time_limit
        self.constraints = constraints
        self.speed = speed
        self.daily_limit = daily_limit
        self.start_point = None

    def dfs(self, current, remaining, path, total_time):
        if total_time > self.daily_limit:
            return

        if len(path) > len(self.best_path):
            self.best_path = deepcopy(path)
            self.best_time = total_time

        for i, point in enumerate(remaining):
            travel_time = Estimate_Time(current, point, self.speed)['estimated_time_minutes']
            self.dfs(
                point,
                remaining[:i] + remaining[i+1:],
                path + [point],
                total_time + travel_time
            )

    def optimize(self, start, available_points):
        self.best_path = []
        self.best_time = 0
        self.dfs(start, available_points, [start], 0)
        return {
            'visited_points': self.best_path,
            'total_time_minutes': round(self.best_time, 2),
            'points_visited': len(self.best_path)
        }

    def multi_day_optimize(self, start):
        self.start_point = start
        remaining_points = deepcopy(self.data)
        daily_results = []
        total_time_used = 0

        while remaining_points and total_time_used < self.global_time_limit:
            # Remove already visited points from previous runs
            current_remaining = [
                p for p in remaining_points
                if not (p['Longitude'] == start['Longitude'] and p['Latitude'] == start['Latitude'])
            ]

            day_result = self.optimize(start, current_remaining)
            visited = day_result['visited_points'][1:]  # exclude start point from removal
            daily_time = day_result['total_time_minutes']

            if not visited:
                break  # can't fit any more points

            # Add day result and update
            daily_results.append({
                'day': len(daily_results) + 1,
                'path': day_result['visited_points'],
                'time': daily_time
            })
            total_time_used += daily_time
            remaining_points = [
                p for p in remaining_points
                if p not in visited
            ]

        return daily_results



In [73]:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]


start_point = {'name': "Start", 'Longitude': 2.9, 'Latitude': 36.75}
optimizer = PlanOptimizerFull(data=data_points, time_limit=2000, speed=60, daily_limit=480)
results = optimizer.multi_day_optimize(start=start_point)

for r in results:
    print(f"\n📅 Day {r['day']}:")
    for i, pt in enumerate(r['path']):
        print(f"  {i+1}. {pt['name']}")
    print(f"⏱️ Estimated Time: {round(r['time'], 2)} minutes")


KeyboardInterrupt: 

In [74]:
from collections import deque
from copy import deepcopy

class PlanOptimizerFull_BFS(object):
    def __init__(self, data, time_limit, constraints=None, speed=40, daily_limit=480):
        self.original_data = deepcopy(data)
        self.data = deepcopy(data)
        self.global_time_limit = time_limit
        self.constraints = constraints
        self.speed = speed
        self.daily_limit = daily_limit
        self.start_point = None

    def bfs(self, start, remaining):
        # BFS initialization
        queue = deque([(start, remaining, [start], 0)])  # (current_point, remaining_points, path, time_spent)
        best_path = []
        best_time = 0

        while queue:
            current, remaining_points, path, total_time = queue.popleft()

            # If the total time exceeds the daily limit, discard this path
            if total_time > self.daily_limit:
                continue

            # Update the best path if necessary
            if len(path) > len(best_path):
                best_path = deepcopy(path)
                best_time = total_time

            # Process each remaining point
            for i, point in enumerate(remaining_points):
                travel_time = Estimate_Time(current, point, self.speed)['estimated_time_minutes']
                new_path = path + [point]
                new_remaining = remaining_points[:i] + remaining_points[i+1:]

                # Push the new state into the queue
                queue.append((point, new_remaining, new_path, total_time + travel_time))

        return best_path, best_time

    def optimize(self, start, available_points):
        best_path, best_time = self.bfs(start, available_points)
        return {
            'visited_points': best_path,
            'total_time_minutes': round(best_time, 2),
            'points_visited': len(best_path)
        }

    def multi_day_optimize(self, start):
        self.start_point = start
        remaining_points = deepcopy(self.data)
        daily_results = []
        total_time_used = 0

        while remaining_points and total_time_used < self.global_time_limit:
            # Remove already visited points from previous runs
            current_remaining = [
                p for p in remaining_points
                if not (p['Longitude'] == start['Longitude'] and p['Latitude'] == start['Latitude'])
            ]

            day_result = self.optimize(start, current_remaining)
            visited = day_result['visited_points'][1:]  # exclude start point from removal
            daily_time = day_result['total_time_minutes']

            if not visited:
                break  # can't fit any more points

            # Add day result and update
            daily_results.append({
                'day': len(daily_results) + 1,
                'path': day_result['visited_points'],
                'time': daily_time
            })
            total_time_used += daily_time
            remaining_points = [
                p for p in remaining_points
                if p not in visited
            ]

        return daily_results


# Example usage:




In [ ]:
# Example usage:
data_points = [
    {'name': 'Point A', 'Longitude': 3.0588, 'Latitude': 36.7538},
    {'name': 'Point B', 'Longitude': 3.062, 'Latitude': 36.75},
    {'name': 'Point C', 'Longitude': 2.944, 'Latitude': 36.766},
    {'name': 'Point D', 'Longitude': 3.1, 'Latitude': 36.76},
    {'name': 'Point E', 'Longitude': 2.889, 'Latitude': 36.712},
    {'name': 'Point F', 'Longitude': 3.11, 'Latitude': 36.765},
    {'name': 'Point G', 'Longitude': 3.15, 'Latitude': 36.755},
    {'name': 'Point H', 'Longitude': 2.95, 'Latitude': 36.775},
    {'name': 'Point I', 'Longitude': 3.07, 'Latitude': 36.78},
    {'name': 'Point J', 'Longitude': 2.97, 'Latitude': 36.79},
    {'name': 'Point K', 'Longitude': 3.05, 'Latitude': 36.74},
    {'name': 'Point L', 'Longitude': 2.93, 'Latitude': 36.72},
    {'name': 'Point M', 'Longitude': 2.91, 'Latitude': 36.71},
    {'name': 'Point N', 'Longitude': 2.88, 'Latitude': 36.70},
    {'name': 'Point O', 'Longitude': 3.16, 'Latitude': 36.77},
    {'name': 'Point P', 'Longitude': 3.19, 'Latitude': 36.76},
    {'name': 'Point Q', 'Longitude': 3.21, 'Latitude': 36.75},
    {'name': 'Point R', 'Longitude': 3.17, 'Latitude': 36.74},
    {'name': 'Point S', 'Longitude': 2.92, 'Latitude': 36.74},
    {'name': 'Point T', 'Longitude': 2.85, 'Latitude': 36.69}
]


start_point = {'name':"Start",'Longitude': 2.9, 'Latitude': 36.75}
optimizer = PlanOptimizerFull_BFS(data=data_points, time_limit=2000, speed=60)
result = optimizer.multi_day_optimize(start=start_point)

# Call after optimization
print_pretty_result(result)


In [ ]:
data_points = [
    {'name': "Point A", 'Latitude': 36.75, 'Longitude': 2.9},
    {'name': "Point B", 'Latitude': 36.76, 'Longitude': 2.91},
    {'name': "Point C", 'Latitude': 36.77, 'Longitude': 2.92},
    {'name': "Point D", 'Latitude': 36.78, 'Longitude': 2.93},
    {'name': "Point E", 'Latitude': 36.79, 'Longitude': 2.94},
    {'name': "Point F", 'Latitude': 36.80, 'Longitude': 2.95},
    # Add more points as needed
]

start_point = {'name': "Start", 'Longitude': 2.9, 'Latitude': 36.75}

# Set the time limit in minutes (8 hours = 480 minutes)
optimizer = PlanOptimizerFull(data=data_points, time_limit=480, speed=20, daily_limit=480)

result = optimizer.optimize(start=start_point, available_points=data_points)

# Call after optimization
print("Optimization Results:")
print(f"Path: {[point['name'] for point in result['visited_points']]}")
print(f"Total Time: {result['total_time_minutes']} minutes")
print(f"Points Visited: {result['points_visited']}")

In [4]:
import random

# Define the bounds for the coordinates
latitude_bounds = (36.69, 36.80)
longitude_bounds = (2.85, 3.21)

# Function to generate random points
def generate_random_points(num_points):
    points = []
    for i in range(num_points):
        latitude = round(random.uniform(latitude_bounds[0], latitude_bounds[1]), 4)
        longitude = round(random.uniform(longitude_bounds[0], longitude_bounds[1]), 4)
        points.append({
            'name': f'Point {chr(65 + i)}',  # Letters A, B, C, ...
            'Longitude': longitude,
            'Latitude': latitude
        })
    return points

# Generate 40 random points
data_points = generate_random_points(40)
data_points


[{'name': 'Point A', 'Longitude': 2.8837, 'Latitude': 36.7517},
 {'name': 'Point B', 'Longitude': 3.1205, 'Latitude': 36.7621},
 {'name': 'Point C', 'Longitude': 3.1159, 'Latitude': 36.7819},
 {'name': 'Point D', 'Longitude': 3.1708, 'Latitude': 36.7603},
 {'name': 'Point E', 'Longitude': 3.0273, 'Latitude': 36.7596},
 {'name': 'Point F', 'Longitude': 2.8925, 'Latitude': 36.7957},
 {'name': 'Point G', 'Longitude': 2.8814, 'Latitude': 36.7426},
 {'name': 'Point H', 'Longitude': 2.9894, 'Latitude': 36.7961},
 {'name': 'Point I', 'Longitude': 2.9942, 'Latitude': 36.7283},
 {'name': 'Point J', 'Longitude': 2.9032, 'Latitude': 36.7599},
 {'name': 'Point K', 'Longitude': 3.0966, 'Latitude': 36.7413},
 {'name': 'Point L', 'Longitude': 3.0491, 'Latitude': 36.7182},
 {'name': 'Point M', 'Longitude': 2.8541, 'Latitude': 36.7165},
 {'name': 'Point N', 'Longitude': 2.8614, 'Latitude': 36.7509},
 {'name': 'Point O', 'Longitude': 2.9594, 'Latitude': 36.7914},
 {'name': 'Point P', 'Longitude': 3.0949

In [7]:
import math
import pulp
import numpy as np

# Helper: Haversine formula to compute distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in km
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    return R * 2 * math.asin(math.sqrt(a))

# Setup
speed_kmph = 60
time_limit_minutes = 480


start_point = {'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}

# Create a list of all points with the start at index 0
points = [start_point] + data_points
N = len(points)

# Compute travel time matrix in minutes
travel_time = np.zeros((N, N))
for i in range(N):
    for j in range(N):
        if i != j:
            dist = haversine(points[i]['Longitude'], points[i]['Latitude'],
                             points[j]['Longitude'], points[j]['Latitude'])
            time = (dist / speed_kmph) * 60  # in minutes
            travel_time[i][j] = time

# Define IP model
model = pulp.LpProblem("Orienteering", pulp.LpMaximize)

# Decision variables
x = pulp.LpVariable.dicts("x", ((i, j) for i in range(N) for j in range(N) if i != j), cat="Binary")
u = pulp.LpVariable.dicts("u", (i for i in range(N)), lowBound=0, upBound=N-1, cat="Continuous")

# Objective: Maximize number of visited points (excluding start point at 0)
model += pulp.lpSum(x[i, j] for i in range(N) for j in range(N) if i != j and j != 0)

# Constraints

# Start point can only go to one point
model += pulp.lpSum(x[0, j] for j in range(1, N)) == 1  # Only one outgoing edge from start

# No inbound to the start point (no point should come back to the start)
model += pulp.lpSum(x[j, 0] for j in range(1, N)) == 0  # No inbound arcs to the start point

# Flow conservation: each point can have at most one incoming and one outgoing edge
for i in range(1, N):  # Exclude start point (index 0)
    model += pulp.lpSum(x[j, i] for j in range(N) if j != i) <= 1  # At most one incoming edge
    model += pulp.lpSum(x[i, j] for j in range(N) if j != i) <= 1  # At most one outgoing edge

# Time constraint (total travel time must be within the limit)
model += pulp.lpSum(x[i, j] * travel_time[i][j] for i in range(N) for j in range(N) if i != j) <= time_limit_minutes

# Subtour elimination (MTZ)
for i in range(1, N):
    for j in range(1, N):
        if i != j:
            model += u[i] - u[j] + (N - 1) * x[i, j] <= N - 2

# Solve the model
model.solve()

# Extract solution path
solution_edges = [(i, j) for i in range(N) for j in range(N) if i != j and x[i, j].varValue == 1]

solution_names = [points[0]['name']]
current = 0
while True:
    next_edges = [j for i, j in solution_edges if i == current]
    if not next_edges:
        break
    current = next_edges[0]
    solution_names.append(points[current]['name'])

print("Visited Points:", solution_names)


Visited Points: ['Start', 'Point Q', 'Point c', 'Point J', 'Point \\', 'Point Y', 'Point D', 'Point T', 'Point Z', 'Point A', 'Point L', 'Point R', 'Point a', 'Point X', 'Point U', 'Point g', 'Point S', 'Point ]', 'Point `', 'Point H', 'Point [', 'Point C', 'Point G', 'Point W', 'Point ^', 'Point f', 'Point _', 'Point d', 'Point V', 'Point e', 'Point N', 'Point E', 'Point b', 'Point h', 'Point I', 'Point B', 'Point O', 'Point K', 'Point F', 'Point M', 'Point P']


In [49]:
import folium
import math
import pulp
import numpy as np

# Helper: Haversine formula to compute distance between two lat/lon points
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Earth radius in km
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon, dlat = lon2 - lon1, lat2 - lat1
    a = (math.sin(dlat / 2) ** 2 +
         math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2)
    return R * 2 * math.asin(math.sqrt(a))

# Plan route for a single day (or time chunk)
def plan_route(points, speed_kmph, time_limit_minutes, visited_points, visit_cost_minutes=15):
    N = len(points)

    # Create subset of unvisited points, ensuring start point is always included
    unvisited_points = [point for point in points if point['name'] not in visited_points and point['name']!='Start']
    unvisited_points.insert(0, points[0])  # Always start at the same location
    N_unvisited = len(unvisited_points)

    # Compute travel time matrix (in minutes)
    travel_time = np.zeros((N_unvisited, N_unvisited))
    for i in range(N_unvisited):
        for j in range(N_unvisited):
            if i != j:
                dist = haversine(unvisited_points[i]['Longitude'], unvisited_points[i]['Latitude'],
                                 unvisited_points[j]['Longitude'], unvisited_points[j]['Latitude'])
                time = (dist / speed_kmph) * 60
                travel_time[i][j] = time

    # Define integer programming model
    model = pulp.LpProblem("Orienteering", pulp.LpMaximize)

    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(N_unvisited) for j in range(N_unvisited) if i != j), cat="Binary")
    u = pulp.LpVariable.dicts("u", (i for i in range(N_unvisited)), lowBound=0, upBound=N_unvisited - 1, cat="Continuous")

    # Objective: maximize number of visits (excluding return to start)
    model += pulp.lpSum(x[i, j] for i in range(N_unvisited) for j in range(N_unvisited) if i != j and j != 0)

    # Constraints
    model += pulp.lpSum(x[0, j] for j in range(1, N_unvisited)) == 1
    model += pulp.lpSum(x[j, 0] for j in range(1, N_unvisited)) == 0

    for i in range(1, N_unvisited):
        model += pulp.lpSum(x[j, i] for j in range(N_unvisited) if j != i) <= 1
        model += pulp.lpSum(x[i, j] for j in range(N_unvisited) if j != i) <= 1

    model += pulp.lpSum(x[i, j] * (travel_time[i][j] + visit_cost_minutes)
                        for i in range(N_unvisited) for j in range(N_unvisited) if i != j) <= time_limit_minutes

    # Subtour elimination (MTZ)
    for i in range(1, N_unvisited):
        for j in range(1, N_unvisited):
            if i != j:
                model += u[i] - u[j] + (N_unvisited - 1) * x[i, j] <= N_unvisited - 2

    model.solve()

    # Extract solution
    solution_edges = [(i, j) for i in range(N_unvisited) for j in range(N_unvisited)
                      if i != j and pulp.value(x[i, j]) == 1]

    # Build ordered route
    solution_order = [unvisited_points[0]['name']]
    current = 0
    while True:
        next_nodes = [j for i, j in solution_edges if i == current]
        if not next_nodes:
            break
        current = next_nodes[0]
        solution_order.append(unvisited_points[current]['name'])

    estimated_time = sum(travel_time[i][j] + visit_cost_minutes for i, j in solution_edges)

    return solution_order, solution_edges, unvisited_points, estimated_time

# Multi-day planning
def plan_multiple_days(points, total_minutes, daily_limit_minutes, speed_kmph):
    visited_points = []
    all_routes = []
    all_edges = []
    all_estimates = []

    num_days = math.ceil(total_minutes / daily_limit_minutes)

    for day in range(num_days):
        route, edges, unvisited_points, estimated_time = plan_route(
            points, speed_kmph=speed_kmph, time_limit_minutes=daily_limit_minutes, visited_points=visited_points
        )

        if len(route) <= 1:  # No progress
            break

        all_routes.append(route)
        all_edges.append(edges)
        all_estimates.append(estimated_time)
        visited_points.extend(route[1:])  # Skip start point

        if len(visited_points) >= len(points) - 1:
            break

    return all_routes, all_edges, all_estimates


In [50]:
import random

# Define the bounds for the coordinates
latitude_bounds = (36.69, 36.80)
longitude_bounds = (2.85, 3.21)

# Function to generate random points
def generate_random_points(num_points):
    points = []
    for i in range(num_points):
        latitude = round(random.uniform(latitude_bounds[0], latitude_bounds[1]), 4)
        longitude = round(random.uniform(longitude_bounds[0], longitude_bounds[1]), 4)
        points.append({
            'name': f'Point {chr(65 + i)}',  # Letters A, B, C, ...
            'Longitude': longitude,
            'Latitude': latitude
        })
    return points

# Generate 40 random points
data_points = generate_random_points(40)



In [58]:
# Parameters
total_time_minutes = 7 * 60 * 10  # 8 hours total
daily_limit_minutes = 7 * 60  # 2 hours per day
speed_kmph = 50  # Average walking speed

# Run the multi-day planner
routes, edges, estimates = plan_multiple_days([{'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}] + data_points, total_time_minutes, daily_limit_minutes, speed_kmph)

# Display results
for day, route in enumerate(routes, 1):
    print(f"Day {day}:")
    print(f"Number of Visits {len(route)-1}:")
    print(" → ".join(route))
    print(f"Estimated time: {estimates[day-1]:.2f} minutes\n")
    


Day 1:
Number of Visits 2:
Start → Point _ → Point `
Estimated time: 412.13 minutes

Day 2:
Number of Visits 1:
Start → Point N
Estimated time: 411.69 minutes

Day 3:
Number of Visits 1:
Start → Point W
Estimated time: 419.51 minutes

Day 4:
Number of Visits 1:
Start → Point h
Estimated time: 418.18 minutes

Day 5:
Number of Visits 3:
Start → Point [ → Point I → Point O
Estimated time: 419.17 minutes

Day 6:
Number of Visits 1:
Start → Point Y
Estimated time: 419.81 minutes

Day 7:
Number of Visits 1:
Start → Point e
Estimated time: 419.72 minutes

Day 8:
Number of Visits 7:
Start → Point M → Point S → Point E → Point c → Point T → Point B → Point a
Estimated time: 415.44 minutes

Day 9:
Number of Visits 23:
Start → Point g → Point Z → Point R → Point ^ → Point D → Point ] → Point V → Point P → Point J → Point Q → Point A → Point X → Point C → Point H → Point f → Point b → Point \ → Point L → Point K → Point G → Point F → Point d → Point U
Estimated time: 419.92 minutes



In [60]:
number_of_visits=0
for i in range(len(routes)):
    number_of_visits+=len(routes[i])-1
print(number_of_visits)
print(len(data_points))

40
40


In [53]:
import folium

def visualize_routes(routes, edges, all_unvisited_points):
    # Map each point name to its coordinates for easy lookup
    point_lookup = {p['name']: (p['Latitude'], p['Longitude']) for p in all_unvisited_points}

    for day, (route, day_edges) in enumerate(zip(routes, edges), 1):
        # Create base map centered on the starting point
        start_lat, start_lon = point_lookup['Start']
        m = folium.Map(location=[start_lat, start_lon], zoom_start=12, tiles="OpenStreetMap")

        # Add all points with markers
        for name in route:
            lat, lon = point_lookup[name]
            folium.Marker([lat, lon], tooltip=name, icon=folium.Icon(color='blue' if name != 'Start' else 'red')).add_to(m)

        # Draw lines between edges
        for i, j in day_edges:
            from_point = all_unvisited_points[i]
            to_point = all_unvisited_points[j]
            folium.PolyLine(
                [(from_point['Latitude'], from_point['Longitude']),
                 (to_point['Latitude'], to_point['Longitude'])],
                color='green', weight=3, opacity=0.8
            ).add_to(m)

        # Save map to HTML
        m.save(f"route_day_{day}.html")
        print(f"Map saved: route_day_{day}.html")


In [61]:
# `data_points` is assumed to be your list of locations (excluding start)
all_unvisited_points = [{'name': 'Start', 'Longitude': 2.9, 'Latitude': 36.75}] + data_points

visualize_routes(routes, edges, all_unvisited_points)


Map saved: route_day_1.html
Map saved: route_day_2.html
Map saved: route_day_3.html
Map saved: route_day_4.html
Map saved: route_day_5.html
Map saved: route_day_6.html
Map saved: route_day_7.html
Map saved: route_day_8.html
Map saved: route_day_9.html


In [63]:
!pip install rest_framework

ERROR: Could not find a version that satisfies the requirement rest_framework (from versions: none)
ERROR: No matching distribution found for rest_framework
